In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark

In [2]:
from pyspark import SparkContext
sc=SparkContext()

In [3]:
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)

In [4]:
from pyspark.sql.functions import col,isnan,when,count,split,length,trim,sum,avg,desc,round
from pyspark.sql.types import (
    StringType, BooleanType, IntegerType, FloatType, DateType
)


In [8]:
bdoriginal=sqlContext.read.format("com.databricks.spark.csv").option("header","true").load("../complaints.csv", inferSchema=True)
sqlContext.registerDataFrameAsTable(bdoriginal,"bd")

In [9]:
bdoriginal.count()

4876519

In [10]:
bdoriginal.show()

+-------------+--------------------+----------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+--------------------------+-------------+--------------------+----------------------------+----------------+------------------+------------+
|Date received|             Product|     Sub-product|               Issue|           Sub-issue|Consumer complaint narrative|Company public response|             Company|State|ZIP code|         Tags|Consumer consent provided?|Submitted via|Date sent to company|Company response to consumer|Timely response?|Consumer disputed?|Complaint ID|
+-------------+--------------------+----------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+--------------------------+-------------+--------------------+----------------------------+----------------+----------------

In [36]:

bd = bdoriginal.withColumn('year', split(bdoriginal['Date received'], '-').getItem(0)) \
       .withColumn('month', split(bdoriginal['Date received'], '-').getItem(1)) \
       .withColumn('day', split(bdoriginal['Date received'], '-').getItem(2))
bd = bd.filter("day is not null and month is not null and year is not null")
bd = bd.filter("Product is not null")
bd = bd.filter(length(col("day")) <3)
bd = bd.filter(length(col("Product")) <20)
bd.show(truncate=False)


bd.count()


+-------------+---------------+--------------------------+-----------------------------------------------------------+--------------------------------------------------------------+----------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------+-----+--------+-----------------------------+--------------------------+-------------+--------------------+----------------------------+----------------+------------------+------------+----+-----+---+
|Date received|Product        |Sub-product               |Issue                                                      |Sub-issue                                                     |Consumer complaint narrative|Company public response                                                                        |Company                               |State|ZIP code|Tags                         |Consumer consent provided?|Submitted via|Date sent to company|Co

1184424

In [38]:
def unificar(df, nombre_archivo):
    df2 = df.repartition(1)
    return df2.write.option("header",True).csv(nombre_archivo)

In [13]:
df = group_item("Product","100")
df.show(df.count())

+----------------+------+----+
|         Product| count| avg|
+----------------+------+----+
| Debt collection|464901|0.39|
|        Mortgage|370360|0.31|
|Credit reporting|140429|0.12|
|     Credit card| 89190|0.08|
|    Student loan| 72929|0.06|
|   Consumer Loan| 31596|0.03|
|     Payday loan|  5543| 0.0|
| Money transfers|  5354| 0.0|
|    Prepaid card|  3819| 0.0|
+----------------+------+----+



In [40]:
df_select = bd.where("Product = 'Mortgage' OR Product = 'Debt collection'  OR Product = 'Credit reporting'  OR Product = 'Credit card' OR Product = 'Student loan' OR Product = 'Consumer Loan'")

In [45]:
df_select = df_select.select("Product","Sub-product","Issue","Company","State","ZIP code","Tags","Submitted via","Consumer disputed?","year","month")
df_select.show()

+---------------+--------------------+--------------------+--------------------+-----+--------+--------------------+-------------+------------------+----+-----+
|        Product|         Sub-product|               Issue|             Company|State|ZIP code|                Tags|Submitted via|Consumer disputed?|year|month|
+---------------+--------------------+--------------------+--------------------+-----+--------+--------------------+-------------+------------------+----+-----+
|       Mortgage|Conventional home...|Trouble during pa...| Selene Holdings LLC|   AZ|   85255|                null|          Web|               N/A|2023|   01|
|Debt collection|    Credit card debt|False statements ...|Dimand Law Office...|   IL|   60652|                null|          Web|               N/A|2023|   02|
|Debt collection|       I do not know|Attempts to colle...|Aldridge Pite Haa...|   GA|   30047|                null|          Web|               N/A|2023|   02|
|       Mortgage|Conventional home

In [46]:
df_select.count()

1169405

In [47]:
def group_item(columna, cantidad):
    
    df = df_select.select(columna)
    df = df.groupBy(columna).count()
    df = df.sort(col("count").desc())
    total = df.agg({'count': 'sum'}).collect()[0][0]
    df = df.withColumn("avg",round(col("count")/total,2))
    df = df.filter("count > " + cantidad)
    return df

In [48]:
df_issue = group_item("Issue","100")
df_issue.show()
df.count()

+--------------------+------+----+
|               Issue| count| avg|
+--------------------+------+----+
|Attempts to colle...|159583|0.14|
|Loan modification...|112307| 0.1|
|Incorrect informa...|102684|0.09|
|Loan servicing, p...| 77332|0.07|
|Written notificat...| 71664|0.06|
|Trouble during pa...| 66596|0.06|
|Cont'd attempts c...| 60665|0.05|
|Communication tac...| 51215|0.04|
|Struggling to pay...| 40198|0.03|
|False statements ...| 37986|0.03|
|Disclosure verifi...| 30790|0.03|
|Took or threatene...| 27657|0.02|
|Dealing with your...| 27227|0.02|
|Applying for a mo...| 19428|0.02|
|Dealing with my l...| 17630|0.02|
|Application, orig...| 17229|0.01|
|Credit reporting ...| 16883|0.01|
|Managing the loan...| 15278|0.01|
|    Billing disputes| 15136|0.01|
|               Other| 14778|0.01|
+--------------------+------+----+
only showing top 20 rows



14

In [51]:
df = group_item("Consumer disputed?","100")
df_pd=df.toPandas()
df.show()


+--------------------+------+----+
|  Consumer disputed?| count| avg|
+--------------------+------+----+
|                  No|497531|0.43|
|                 N/A|395053|0.34|
|                null|138338|0.12|
|                 Yes|120628| 0.1|
|Closed with expla...|  2876| 0.0|
|                 Web|  1999| 0.0|
|    Consent provided|  1635| 0.0|
| or delete the it...|   804| 0.0|
|Closed with non-m...|   334| 0.0|
|Closed with monet...|   162| 0.0|
|       Servicemember|   135| 0.0|
|                  CA|   118| 0.0|
|Company has respo...|   115| 0.0|
+--------------------+------+----+



In [55]:
print(df_pd.to_csv("../consumer_disputed.csv"))  

None


In [63]:
df_year = group_item("year","100")
df_year = df_year.withColumnRenamed("count","total_quejas")
df_pd=df_year.toPandas()
df_year.show(df_year.count())

+----+------------+----+
|year|total_quejas| avg|
+----+------------+----+
|2016|      164753|0.14|
|2015|      146017|0.12|
|2014|      135039|0.12|
|2017|      122946|0.11|
|2021|      101007|0.09|
|2013|       94074|0.08|
|2022|       90721|0.08|
|2018|       84496|0.07|
|2020|       82972|0.07|
|2019|       76294|0.07|
|2012|       60160|0.05|
|2023|        8390|0.01|
|2011|        2536| 0.0|
+----+------------+----+



In [64]:
print(df_pd.to_csv("../year_complaints.csv"))  

None


In [65]:
df_companies = group_item("company","100")
df_companies.show()

+--------------------+------+----+
|             company| count| avg|
+--------------------+------+----+
|                null|133094|0.11|
|BANK OF AMERICA, ...| 56506|0.05|
|       EQUIFAX, INC.| 51622|0.04|
|WELLS FARGO & COM...| 50429|0.04|
|Experian Informat...| 48879|0.04|
|TRANSUNION INTERM...| 43716|0.04|
|JPMORGAN CHASE & CO.| 36704|0.03|
|      CITIBANK, N.A.| 32133|0.03|
|Ocwen Financial C...| 28919|0.02|
|Navient Solutions...| 28835|0.02|
|CAPITAL ONE FINAN...| 22204|0.02|
|NATIONSTAR MORTGA...| 22188|0.02|
|PORTFOLIO RECOVER...| 16228|0.01|
|ENCORE CAPITAL GR...| 15641|0.01|
| SYNCHRONY FINANCIAL| 14988|0.01|
|Ditech Financial LLC| 12942|0.01|
|        U.S. BANCORP| 10288|0.01|
|SELECT PORTFOLIO ...| 10079|0.01|
|           AES/PHEAA|  9691|0.01|
|                 ERC|  8968|0.01|
+--------------------+------+----+
only showing top 20 rows



In [72]:
df_disputas = df_select.withColumnRenamed("Consumer disputed?","disputa")
df_disputas = df_banco.filter("disputa ='Yes'")


+---------------+--------------------+--------------------+--------------------+--------------------+--------+-----+----------------+-------+----+-----+
|        Product|         Sub-product|               Issue|             Company|               State|ZIP code| Tags|   Submitted via|disputa|year|month|
+---------------+--------------------+--------------------+--------------------+--------------------+--------+-----+----------------+-------+----+-----+
|Debt collection|       I do not know|Communication tac...|Company has respo...|Radius Global Sol...|      MD|20895|Consent provided|    Yes|2023|   01|
|Debt collection|          Other debt|Attempts to colle...|Company disputes ...|The Collection Ag...|      NE|68310|Consent provided|    Yes|2022|   12|
|Debt collection|          Other debt|Written notificat...|Company believes ...|  LJ Ross Associates|      IL|60634|Consent provided|    Yes|2022|   12|
|       Mortgage|Conventional home...|Trouble during pa...|                null|Cr

In [74]:
def group_item_v2(columna, cantidad, df_i):
    
    df = df_i.select(columna)
    df = df.groupBy(columna).count()
    df = df.sort(col("count").desc())
    total = df.agg({'count': 'sum'}).collect()[0][0]
    df = df.withColumn("avg",round(col("count")/total,2))
    df = df.filter("count > " + cantidad)
    return df

In [76]:
df_disputas.count()

120628

In [75]:
df_disputas_companies = group_item_v2("company","50",df_disputas)
df_disputas_companies.show(df_disputas_companies.count())

+--------------------+-----+----+
|             company|count| avg|
+--------------------+-----+----+
|BANK OF AMERICA, ...|11322|0.09|
|       EQUIFAX, INC.| 8831|0.07|
|WELLS FARGO & COM...| 8291|0.07|
|JPMORGAN CHASE & CO.| 6586|0.05|
|      CITIBANK, N.A.| 5369|0.04|
|Ocwen Financial C...| 5235|0.04|
|TRANSUNION INTERM...| 4922|0.04|
|Experian Informat...| 4747|0.04|
|NATIONSTAR MORTGA...| 3362|0.03|
|CAPITAL ONE FINAN...| 2903|0.02|
|Navient Solutions...| 2632|0.02|
|                null| 2516|0.02|
|Ditech Financial LLC| 2190|0.02|
|AMERICAN EXPRESS ...| 1729|0.01|
| SYNCHRONY FINANCIAL| 1652|0.01|
|        U.S. BANCORP| 1624|0.01|
|ENCORE CAPITAL GR...| 1313|0.01|
|HSBC NORTH AMERIC...| 1299|0.01|
|SELECT PORTFOLIO ...| 1234|0.01|
|       DISCOVER BANK| 1047|0.01|
|PORTFOLIO RECOVER...|  995|0.01|
|       PNC Bank N.A.|  958|0.01|
|Company has respo...|  954|0.01|
|       Seterus, Inc.|  772|0.01|
|SUNTRUST BANKS, INC.|  728|0.01|
|Santander Consume...|  726|0.01|
|           AE

In [77]:
df_disputas_companies=df_disputas_companies.withColumnRenamed("count","total_disputas")
df_disputas_companies=df_disputas_companies.withColumnRenamed("avg","porc_disputas")
df_disputas_companies.show()

+--------------------+--------------+-------------+
|             company|total_disputas|porc_disputas|
+--------------------+--------------+-------------+
|BANK OF AMERICA, ...|         11322|         0.09|
|       EQUIFAX, INC.|          8831|         0.07|
|WELLS FARGO & COM...|          8291|         0.07|
|JPMORGAN CHASE & CO.|          6586|         0.05|
|      CITIBANK, N.A.|          5369|         0.04|
|Ocwen Financial C...|          5235|         0.04|
|TRANSUNION INTERM...|          4922|         0.04|
|Experian Informat...|          4747|         0.04|
|NATIONSTAR MORTGA...|          3362|         0.03|
|CAPITAL ONE FINAN...|          2903|         0.02|
|Navient Solutions...|          2632|         0.02|
|                null|          2516|         0.02|
|Ditech Financial LLC|          2190|         0.02|
|AMERICAN EXPRESS ...|          1729|         0.01|
| SYNCHRONY FINANCIAL|          1652|         0.01|
|        U.S. BANCORP|          1624|         0.01|
|ENCORE CAPI

In [78]:
df_disputas_year = group_item_v2("year","50",df_disputas)
df_disputas_year.show(df_disputas_year.count())

+----+-----+----+
|year|count| avg|
+----+-----+----+
|2014|26375|0.22|
|2015|25875|0.21|
|2016|24566| 0.2|
|2013|19891|0.16|
|2012|13849|0.11|
|2017| 7786|0.06|
|2011|  573| 0.0|
|2021|  389| 0.0|
|2018|  376| 0.0|
|2020|  342| 0.0|
|2019|  329| 0.0|
|2022|  275| 0.0|
+----+-----+----+



In [79]:
df_pd=df_disputas_year.toPandas()
print(df_pd.to_csv("../disputas_year_complaints.csv"))  

None


In [23]:
df = group_item("state","100")
df.show(df.count())

+--------------------+------+----+
|               state| count| avg|
+--------------------+------+----+
|                null|147867|0.12|
|                  CA|133889|0.11|
|                  FL|100712|0.09|
|                  TX| 92540|0.08|
|                  NY| 64954|0.05|
|                  GA| 55547|0.05|
|                  NJ| 38028|0.03|
|                  IL| 36451|0.03|
|                  PA| 35915|0.03|
|                  VA| 30952|0.03|
|                  OH| 30820|0.03|
|                  NC| 29962|0.03|
|                  MD| 29962|0.03|
|                  MI| 25087|0.02|
|                  AZ| 21813|0.02|
|                  WA| 19372|0.02|
|                  MA| 18860|0.02|
|                  TN| 17263|0.01|
|                  SC| 16338|0.01|
|                  CO| 16204|0.01|
|                  MO| 14018|0.01|
|                  NV| 12913|0.01|
|                  LA| 12906|0.01|
|                  AL| 12396|0.01|
|                  IN| 11700|0.01|
|                  C

In [34]:
df_consulta = df_select #.filter("product = 'Mortgage'")
df_consulta = df_consulta.groupBy("company").count()
df_consulta = df_consulta.filter("count > 100")
df_consulta.sort(col("count").desc()).show(df_consulta.count())

+--------------------+------+
|             company| count|
+--------------------+------+
|                null|133094|
|BANK OF AMERICA, ...| 56506|
|       EQUIFAX, INC.| 51622|
|WELLS FARGO & COM...| 50429|
|Experian Informat...| 48879|
|TRANSUNION INTERM...| 43716|
|JPMORGAN CHASE & CO.| 36704|
|      CITIBANK, N.A.| 32133|
|Ocwen Financial C...| 28919|
|Navient Solutions...| 28835|
|CAPITAL ONE FINAN...| 22204|
|NATIONSTAR MORTGA...| 22188|
|PORTFOLIO RECOVER...| 16228|
|ENCORE CAPITAL GR...| 15641|
| SYNCHRONY FINANCIAL| 14988|
|Ditech Financial LLC| 12942|
|        U.S. BANCORP| 10288|
|SELECT PORTFOLIO ...| 10079|
|           AES/PHEAA|  9691|
|                 ERC|  8968|
|Resurgent Capital...|  8679|
|AMERICAN EXPRESS ...|  7788|
|       DISCOVER BANK|  7578|
|       PNC Bank N.A.|  6824|
|HSBC NORTH AMERIC...|  6776|
|TRANSWORLD SYSTEM...|  6295|
|Convergent Resour...|  6144|
|   I.C. System, Inc.|  6138|
|Specialized Loan ...|  5705|
|Shellpoint Partne...|  5472|
|CCS Finan

In [129]:
print(unificar(df_consulta,"company_counts"))

None
